In [1]:
import docplex.mp.model as cpx

# The set of operations
I = 3
# The set of machines
J = 3
# The set of hanling machines
L = 2
# set of products
K = 4

# Define the parameters

c = [[6, 13, 8], 
     [3, 19, 4], 
     [11, 9, 7]]

h = [[9, 4],
    [12, 15],
    [10, 5],
    [20, 16]]

cprim = [210000, 260000, 300000]

hprim = [80000, 120000]

B = 11000000

t = [[10/6, 10/13, 10/8],
    [10/3, 10/19, 10/4],
    [10/11, 10/9, 10/7]]

#IF data is too much,could use this roup to get t
#t = []
#for i in range(I):
#    t.append([])
#    for j in range(J):
#        t[i].append(10/c[i][j])



s = [[1/9, 1/4],
    [1/12, 1/15],
    [1/10, 1/5],
    [1/20, 1/16]]

T = [6000, 7000, 5000]

D = [2000, 1500, 1800, 1000]

# create the cplex model

MSP_model = cpx.Model('MIP Model')


# Time limit
MSP_model.parameters.timelimit.set(3600)
# Decision variables

M_range = [(j) for j in range(J)]
MH_range = [(l) for l in range(L)]
x_range = [(i,j) for i in range(I) for j in range(J)]
y_range = [(k,l) for k in range(K) for l in range(L)]

M = MSP_model.continuous_var_dict(M_range, name = "M")
MH = MSP_model.integer_var_dict(MH_range, name = "MH")
x = MSP_model.integer_var_dict(x_range, name = "x")
y = MSP_model.integer_var_dict(y_range, name = "y")

#Q = MSP_model.integer_var(name = "y")

# Objective funtion

objective = MSP_model.sum( cprim[j] * M[j] for j in range(J)) + MSP_model.sum( hprim[l] * MH[l] for l in range(L)) + MSP_model.sum( c[i][j] * x[i,j] for i in range(I) for j in range(J)) + MSP_model.sum( h[k][l] * y[k,l] for k in range(K) for l in range(L))


# constraints 
for i in range(I):
    MSP_model.add_constraint(MSP_model.sum(x[i,j] for j in range(J)) >= T[i])

for k in range(K):
    MSP_model.add_constraint(MSP_model.sum(y[k,l] for l in range(L)) >= D[k])
    
for j in range(J):
    MSP_model.add_constraint(MSP_model.sum(t[i][j] * x[i,j] for i in range (I)) <= 480 * M[j])

for l in range(L):
    MSP_model.add_constraint(MSP_model.sum(s[k][l] * y[k,l] for k in range (K)) <= 480 * MH[l])

MSP_model.add_constraint(MSP_model.sum( cprim[j] * M[j] for j in range(J)) + MSP_model.sum( hprim[l] * MH[l] for l in range(L)) <= B)

# Solve the model
MSP_model.minimize(objective)

MSP_model.solve(log_output = True)

# Some useful funtions

Obj_value = MSP_model.objective_value
LB = MSP_model.solve_details.best_bound
Gap = MSP_model.solve_details.mip_relative_gap
CPU_time = MSP_model.get_solve_details().time
print('Objective value' ,Obj_value)
print('Lower bound' ,LB)
print('Gap' ,Gap)
print('time' , CPU_time)


Version identifier: 20.1.0.1 | 2021-12-07 | 9dfdf6686
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Reduced MIP has 13 rows, 22 columns, and 44 nonzeros.
Reduced MIP has 0 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.02 ticks)
Tried aggregator 2 times.
Aggregator did 4 substitutions.
Reduced MIP has 9 rows, 18 columns, and 36 nonzeros.
Reduced MIP has 0 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
Reduced MIP has 9 rows, 18 columns, and 36 nonzeros.
Reduced MIP has 0 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
 

In [2]:
for j in range(J):
    print((j), " : " ,M[j].solution_value)

0  :  9.469696969696969
1  :  17.29082321187584
2  :  0


In [3]:
for l in range(L):
    print((l), " : " ,MH[l].solution_value)

0  :  2.0
1  :  0


In [4]:
for i in range(I):
    for j in range(J):
        print((i,j), " : " ,x[i,j].solution_value)

(0, 0)  :  0
(0, 1)  :  6000.0
(0, 2)  :  0
(1, 0)  :  0
(1, 1)  :  7000.0
(1, 2)  :  0
(2, 0)  :  5000.0
(2, 1)  :  0
(2, 2)  :  0


In [5]:
for k in range(K):
    for l in range(L):
        print((k,l), " : " ,y[k,l].solution_value)

(0, 0)  :  2000.0
(0, 1)  :  0
(1, 0)  :  1500.0
(1, 1)  :  0
(2, 0)  :  1800.0
(2, 1)  :  0
(3, 0)  :  1000.0
(3, 1)  :  0
